# HW 03

Charlie Perez (cwp5xyj)

## Part 1: Creating TOKENS and VOCAB

In [1]:
import pandas as pd
import numpy as np

import configparser
config = configparser.ConfigParser()
config.read("../../../env.ini")
data_dir = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']

data_dir, output_dir

('/home/cwp5xyj/Documents/MSDS/DS5001/data',
 '/home/cwp5xyj/Documents/MSDS/DS5001/output')

In [48]:
frankenstein_text = f"{data_dir}/pg42324.txt"
OHCO = ['chap_id','para_num','sent_num','token_num']

In [35]:
# hmmmm okkk let's import textimporter

import sys
import os

sys.path.append(os.path.abspath('/home/cwp5xyj/Documents/MSDS/DS5001/repo-main/lessons/lib'))
from textimporter import TextImporter

In [39]:
ohco_pats = [('chap', r'^(?:PREFACE|LETTER|CHAPTER)\b', 'm')]
clip_pats = [r'START', r'END']
frankenstein = TextImporter(src_file=frankenstein_text, ohco_pats=ohco_pats, clip_pats=clip_pats)
frankenstein.import_source().parse_tokens()
frankenstein.TOKENS.head()

Importing  /home/cwp5xyj/Documents/MSDS/DS5001/data/pg42324.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone ^(?:PREFACE|LETTER|CHAPTER)\b
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by delimitter [.?!;:]+
Parsing OHCO level 3 token_num by delimitter [\s',-]+


token_str term_str
chap_id para_num sent_num token_num                   
1       0        0        0               The      the
                          1             event    event
                          2                on       on
                          3             which    which
                          4              this     this

In [40]:
frankenstein.TOKENS.tail()

token_str  term_str
chap_id para_num sent_num token_num                    
31      82       1        10             lost      lost
                          11               in        in
                          12         darkness  darkness
                          13              and       and
                          14         distance  distance

In [41]:
# looks good

TOKENS = frankenstein.TOKENS

In [44]:
frankenstein.extract_vocab()
frankenstein.VOCAB.head()

,n,n_chars,p,s,i,h
term_str,,,,,,
the,4248,3,0.055695,17.955038,4.166317,0.232042
and,2991,3,0.039214,25.500836,4.672473,0.183228
i,2858,1,0.037471,26.687544,4.738095,0.177540
of,2683,2,0.035176,28.428252,4.829253,0.169875
to,2118,2,0.027769,36.011804,5.170398,0.143575


In [45]:
VOCAB = frankenstein.VOCAB

In [50]:
VOCAB['modified_term_str'] = VOCAB.index
VOCAB.loc[(VOCAB.n == 1) & (VOCAB.n_chars < 3), 'modified_term_str'] = "<UNK>"

In [53]:
VOCAB[VOCAB.modified_term_str=='<UNK>']

,n,n_chars,p,s,i,h,modified_term_str
term_str,,,,,,,
2d,1,2,0.000013,76273.0,16.218885,0.000213,<UNK>
3,1,1,0.000013,76273.0,16.218885,0.000213,<UNK>
du,1,2,0.000013,76273.0,16.218885,0.000213,<UNK>
la,1,2,0.000013,76273.0,16.218885,0.000213,<UNK>
dr,1,2,0.000013,76273.0,16.218885,0.000213,<UNK>
19,1,2,0.000013,76273.0,16.218885,0.000213,<UNK>
w,1,1,0.000013,76273.0,16.218885,0.000213,<UNK>
ne,1,2,0.000013,76273.0,16.218885,0.000213,<UNK>
er,1,2,0.000013,76273.0,16.218885,0.000213,<UNK>


In [56]:
TOKENS['modified_term_str'] = TOKENS.term_str.map(VOCAB.modified_term_str)

## Part 2: making N-grams

In [58]:
ngrams = 3
widx = [f"w{i}" for i in range(ngrams)]

def token_to_padded(token, grouper=['sent_num'], term_str='term_str'):
    ohco = token.index.names # We preserve these since they get lost in the shuffle
    padded = token.groupby(grouper)\
        .apply(lambda x: '<s> ' + ' '.join(x[term_str]) + ' </s>')\
        .apply(lambda x: pd.Series(x.split()))\
        .stack().to_frame('term_str')
    padded.index.names = ohco
    return padded

PADDED = token_to_padded(TOKENS, grouper=OHCO[:3], term_str='modified_term_str')
PADDED.head()

term_str
chap_id para_num sent_num token_num         
1       0        0        0              <s>
                          1              the
                          2            event
                          3               on
                          4            which

In [59]:
def padded_to_ngrams(padded, grouper=['sent_num'], n=2):
    
    ohco = padded.index.names
    ngrams = padded.groupby(grouper)\
        .apply(lambda x: pd.concat([x.shift(0-i) for i in range(n)], axis=1))\
        .reset_index(drop=True)
    ngrams.index = padded.index
    ngrams.columns = widx

    # ngrams = pd.concat([padded.shift(0-i) for i in range(n)], axis=1)
    # ngrams.index.name = 'ngram_num'
    # ngrams.columns = widx
    # ngrams = ngrams.fillna('<EOF>')
    
    return ngrams

In [61]:
NGRAMS = padded_to_ngrams(PADDED, OHCO[:3], ngrams)

In [63]:
NGRAMS.head()

w0     w1       w2
chap_id para_num sent_num token_num                       
1       0        0        0            <s>    the    event
                          1            the  event       on
                          2          event     on    which
                          3             on  which     this
                          4          which   this  fiction

In [64]:
# looking good, looking good

def ngrams_to_models(ngrams):
    global widx
    n = len(ngrams.columns)
    model = [None for i in range(n)]
    for i in range(n):
        if i == 0:
            model[i] = ngrams.value_counts('w0').to_frame('n')
            model[i]['p'] = model[i].n / model[i].n.sum()
            model[i]['i'] = np.log2(1/model[i].p)
        else:
            model[i] = ngrams.value_counts(widx[:i+1]).to_frame('n')    
            model[i]['cp'] = model[i].n / model[i-1].n
            model[i]['i'] = np.log2(1/model[i].cp)
        model[i] = model[i].sort_index()
    return model

In [65]:
M = ngrams_to_models(NGRAMS)

In [68]:
M[0].head()

,n,p,i
w0,,,
1,2,0.000023,15.396504
11th,2,0.000023,15.396504
12th,2,0.000023,15.396504
13th,1,0.000012,16.396504
17,10,0.000116,13.074576


In [70]:
M[1].head(10)

n   cp    i
w0   w1                
1    </s>   2  1.0  0.0
11th 17     1  0.5  1.0
     the    1  0.5  1.0
12th 17     1  0.5  1.0
     </s>   1  0.5  1.0
13th 17     1  1.0  0.0
17   </s>  10  1.0  0.0
1816 in     1  1.0  0.0
1817 </s>   1  1.0  0.0
18th 17     2  1.0  0.0

In [74]:
M[2].iloc[200:220]

n        cp         i
w0  w1  w2                                 
<s> and i            62  0.147268  2.763480
        if           15  0.035629  4.810786
        immediately   1  0.002375  8.717676
        in           11  0.026128  5.258245
        indeed        2  0.004751  7.717676
        instead       2  0.004751  7.717676
        it            8  0.019002  5.717676
        leaning       1  0.002375  8.717676
        like          1  0.002375  8.717676
        may           1  0.002375  8.717676
        my           18  0.042755  4.547751
        no            1  0.002375  8.717676
        not           1  0.002375  8.717676
        now          11  0.026128  5.258245
        often         1  0.002375  8.717676
        oh            1  0.002375  8.717676
        on            7  0.016627  5.910322
        once          3  0.007126  7.132714
        one           1  0.002375  8.717676
        partly        1  0.002375  8.717676

## Part 3: The Questions

### Q1: what are six words that precede 'monster'?

In [78]:
# using the bigram model, querying the second word (w1)
query_1 = M[1].query('w1=="monster"')
query_1

,,n,cp,i
w0,w1,,,
<s>,monster,1,0.000193,12.336507
a,monster,3,0.002141,8.867279
abhorred,monster,1,0.083333,3.584963
detestable,monster,1,0.500000,1.000000
gigantic,monster,1,0.166667,2.584963
hellish,monster,1,0.142857,2.807355
hideous,monster,1,0.090909,3.459432
miserable,monster,1,0.015385,6.022368
the,monster,20,0.004708,7.730640


I could get a bit more creative and extract the words in `w0`, but I think writing it out will suffice. There appear to be only 6 words that are not stop words or sentence boundary markers that precede "monster": **"abhorred", "detestable", "gigantic", "hellish", "hideous", and "miserable".** The poor guy.

### Q2: List the following sentences in order of bigram perplexity:

`The monster is on the ice.`

`Flowers are happy things.`

`I have never seen the aurora borealis.`

`He never knew the love of a family.`

In [80]:
def sentence_to_token(sent_list, file=True):
    
    # Convert list of sentences to dataframe
    if file:
        S = pd.read_csv("test_sentences.txt", header=None, names=['sent_str'])
    else:
        S = pd.DataFrame(sent_list, columns=['sent_str'])
    S.index.name = 'sent_num'
    
    # Convert dataframe of sentences to TOKEN with normalized terms
    K = S.sent_str.apply(lambda x: pd.Series(x.split())).stack().to_frame('token_str')
    K['term_str'] = K.token_str.str.replace(r"[\W_]+", "", regex=True).str.lower()
    K.index.names = ['sent_num', 'token_num']
    
    return S, K

In [82]:
sents = ['The monster is on the ice.',
         'Flowers are happy things.', 
         'I have never seen the aurora borealis.',
         'He never knew the love of a family.']
TEST_SENTS, TEST_TOKENS = sentence_to_token(sents, file=False)
TEST_SENTS

,sent_str
sent_num,
0,The monster is on the ice.
1,Flowers are happy things.
2,I have never seen the aurora borealis.
3,He never knew the love of a family.


In [84]:
TEST_PADDED = token_to_padded(TEST_TOKENS)
TEST_PADDED.head()

term_str
sent_num token_num         
0        0              <s>
         1              the
         2          monster
         3               is
         4               on

In [85]:
TEST_NGRAMS = padded_to_ngrams(TEST_PADDED, 'sent_num', ngrams)

In [86]:
def test_model(model, ngrams, sents):
    
    global widx
    
    assert len(model) == len(ngrams.columns)
    
    n = len(model)
    ohco = ngrams.index.names
    
    R = []
    for i in range(n):
        T = ngrams.merge(M[i], on=widx[:i+1], how='left')
        T.index = ngrams.index
        T = T.reset_index().set_index(ohco + widx).i #.to_frame(f"i{i}")
        
        # This how we handle unseen combos
        T[T.isna()] = T.max()
        R.append(T.to_frame(f"i{i}"))
                
    return pd.concat(R, axis=1)

In [87]:
R = test_model(M,TEST_NGRAMS, TEST_SENTS)

In [88]:
def compute_perplexity(results, test_sents, n=3):
    for i in range(n):
        test_sents[f"pp{i}"] = np.exp2(results.groupby('sent_num')[f"i{i}"].mean())
    return test_sents

In [89]:
PP = compute_perplexity(R, TEST_SENTS)
PP

,sent_str,pp0,pp1,pp2
sent_num,,,,
0,The monster is on the ice.,116.015855,81.183632,70.041494
1,Flowers are happy things.,590.254971,538.146514,187.000000
2,I have never seen the aurora borealis.,341.992368,139.094175,82.690599
3,He never knew the love of a family.,171.277620,137.566400,65.905218


In [93]:
PP.sort_values('pp1', ascending=True)['sent_str']

sent_num
0                The monster is on the ice.
3       He never knew the love of a family.
2    I have never seen the aurora borealis.
1                 Flowers are happy things.
Name: sent_str, dtype: object

### Q3: Exploring relationships between bigram pairs

In [99]:
M[1].head()

n   cp    i
w0   w1               
1    </s>  2  1.0  0.0
11th 17    1  0.5  1.0
     the   1  0.5  1.0
12th 17    1  0.5  1.0
     </s>  1  0.5  1.0

In [101]:
# M[1]['n'].unstack()
mat = M[1]['n'].unstack()
mat.loc[['he','she'],['said','heard']]

w1,said,heard
w0,,
he,21.0,5.0
she,3.0,3.0


So, more males than females in the text, and males are often talking than listening in terms of relative percentages (21/26 for males versus 3/6 for females). But it is a fairly small sample size, particularly for females.

### Q4: Generate 20 sentences

In [116]:
def generate_text(M, n=250):
    
    if len(M) < 3:
        raise ValueError("Must have trigram model generated.")
    
    # Start list of words
    first_word = M[1].loc['<s>'].sample(weights='cp').index[0]
    
    words = ['<s>', first_word]
    
    for i in range(n):
        
        bg = tuple(words[-2:])

        # Try trigram model
        try:
            next_word = M[2].loc[bg].sample(weights='cp').index[0]

        # If not found in model, back off ...
        except KeyError as e1:
            try:
                # Get the last word in the bigram
                ug = bg[1]
                next_word = M[1].loc[ug].sample(weights='cp').index[0]
            
            except KeyError as e2:
                next_word = M[0].sample(weights='p').index[0]
                
        words.append(next_word)
    
    
    text = ' '.join(words[2:])
    return text

In [117]:
# setting seed seems to do nothing here. it generates different things each time
# so what we're gonna do is edit generate_text() to return `text`
# then make the text too long and then cut it
text = generate_text(M, 400)

In [118]:
print('\n\n'.join([str(i+1) + ' ' + line.replace('<s>','')\
    .strip().upper() for i, line in enumerate(text.split('</s>')) if i < 20]))

1 MEANS OF EXTRACTING IT

2 ME AND NOW THAT FALSEHOOD LIES HEAVIER AT MY FEET

3 CONCERNING MY STUDIES DISCONTENTED AND UNSATISFIED

4 

5 NOT KNOW THE NAMES THAT WERE BEGINNING TO CONJECTURE THAT SOME FORTUNATE CHANCE HAD INTERVENED TO PREVENT HER

6 LIFE AND FIXED THEMSELVES IN THEIR SECONDARY AND TERTIARY GRADES WERE UTTERLY UNKNOWN TO THEM BEEN FOR EVER

7 OF THEMSELVES YOU KNOW NOT WHETHER THE RESULT OF MY BEING IS WOUND TO ITS CLOSE

8 PURSUE AND DESTROY THE SAVIOUR OF HIS MACHINATIONS

9 ALTHOUGH UNHAPPY HE IS THUS NOBLE AND GODLIKE

10 A TINGLING LONG LOST SENSE OF MORTAL AGONY CREPT OVER ME AND MY VIRTUES WILL NECESSARILY ARISE WHEN I NEED NOT HAVE BEEN PASSED IN CONSIDERABLE SECLUSION

11 WAFTED TO A HEAVY STORM OF RAIN DESCENDED

12 HER MOTHER WAS DEAD BUT WE ARRIVED IN A DETERMINATION OF NOT CREATING YOU A COMPANION IN THE WORKS OF THE DEED

13 AT LENGTH HID AND I CEASED TO REMONSTRATE

14 

15 ASCERTAINING THE SECRET TO ELIZABETH THIS STRUGGLE MAY OCCASION THE POIGNANT MIS

There are a couple of blank lines, but this seems altogether acceptable.

### Q5: Compute the redundancy *R* for each of the n-gram models

We know that $log{\frac{1}{x}} = -log{x}$. So, I think that all I need to do to calculate H for each is this:

In [135]:
def find_entropy(model, unigram=True):
    # unigram is to signify whether unigram or bi/tri
    if unigram == False:
        mle = model['n'] / model['n'].sum()
    else:
        mle = model['p']
    entropy = -np.sum(mle*np.log2(mle))
    return entropy

In [136]:
H_uni = find_entropy(M[0])
H_bi = find_entropy(M[1], False)
H_tri = find_entropy(M[2], False)

In [137]:
Hmax_uni = np.log2(len(M[0].index))
Hmax_bi = np.log2(len(M[0].index) ** 2)
Hmax_tri = np.log2(len(M[0].index) ** 3)

In [138]:
R_uni = 1 - (H_uni / Hmax_uni)
R_bi = 1 - (H_bi / Hmax_bi)
R_tri = 1 - (H_tri / Hmax_tri)

In [139]:
R_uni, R_bi, R_tri

(0.3085563849837749, 0.44661619331888225, 0.5876206389812256)

*R* appears to increase as the choice of n-gram increases in length. This makes sense given what we know - the unigram model will be the most random while, because every sentence that we used to create our model was written in the same style, that style will become more important for the bi- and tri-gram models.